## Describe your model -> fine-tuned GPT-3.5
By Matt Shumer (https://twitter.com/mattshumer_)

The goal of this notebook is to experiment with a new way to make it very easy to build a task-specific model for your use-case.

First, use the best GPU available (go to Runtime -> change runtime type)

To create your model, just go to the first code cell, and describe the model you want to build in the prompt. Be descriptive and clear.

Select a temperature (high=creative, low=precise), and the number of training examples to generate to train the model. From there, just run all the cells.

You can change the model you want to fine-tune by changing `model_name` in the `Define Hyperparameters` cell.

#Data generation step

Write your prompt here. Make it as descriptive as possible!

Then, choose the temperature (between 0 and 1) to use when generating data. Lower values are great for precise tasks, like writing code, whereas larger values are better for creative tasks, like writing stories.

Finally, choose how many examples you want to generate. The more you generate, a) the longer it takes and b) the more expensive data generation will be. But generally, more examples will lead to a higher-quality model. 100 is usually the minimum to start.

In [1]:
prompt = "A model that takes in a puzzle-like reasoning-heavy and specific questions in English for a college named Claremont Graduate University (CGU), and responds with a well-reasoned, accurate, step-by-step thought out response in English."
temperature = .4
number_of_examples = 150

Run this to generate the dataset.

In [2]:
!pip install openai tenacity

In [2]:
prev_examples = [
    "Do you have online courses available?-----------Yes, we offer a wide range of online courses across various disciplines, allowing students to learn at their own pace and convenience.",
    "What financial aid options are available for students?-----------Students have access to several financial aid options, including scholarships, grants, loans, and work-study programs, depending on their eligibility.",
    "Can international students apply for financial aid?-----------International students can apply for specific scholarships and aid programs. We recommend contacting our financial aid office for more information on eligibility.",
    "How can I apply for a scholarship?-----------You can apply for scholarships through our university's admissions portal. Some scholarships may require additional essays or documentation.",
    "Is campus housing available for first-year students?-----------Yes, campus housing is available and typically guaranteed for first-year students. We encourage early application to secure a spot in our residence halls.",
    "What is the student-to-faculty ratio?-----------Our student-to-faculty ratio is 10:1, ensuring that students receive personalized attention and support in their academic endeavors.",
    "Are there research opportunities for undergraduates?-----------Yes, undergraduates have numerous opportunities to engage in research projects across various departments, often working closely with faculty members.",
    "What career services are offered to graduates?-----------Our career services include resume and cover letter assistance, interview preparation, job search strategies, and networking opportunities with alumni and potential employers."
]


In [3]:
import os
import openai
import random
from tenacity import retry, stop_after_attempt, wait_exponential

# Load the OpenAI API key from an environment variable first, then fallback to direct setting for this example
openai.api_key = os.getenv("OPENAI_API_KEY")

N_RETRIES = 3

@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def generate_example(prompt, prev_examples, temperature=.5):
    messages = [{
        "role": "system",
        "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\nresponse_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
    }]

    # Add previous examples if there are any
    
    if prev_examples:
        if len(prev_examples) > 8:
            prev_examples = random.sample(prev_examples, 8)
        for example in prev_examples:
            messages.append({"role": "assistant", "content": example})

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
        max_tokens=1000,
    )

    # Correctly access the message content using dot notation
    return response.choices[0].message.content

# Generate examples
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print("\nGenerated Examples:")
for example in prev_examples:
    print(example)


Generating example 0
Generating example 1
Generating example 2
Generating example 3
Generating example 4
Generating example 5
Generating example 6
Generating example 7
Generating example 8
Generating example 9
Generating example 10
Generating example 11
Generating example 12
Generating example 13
Generating example 14
Generating example 15
Generating example 16
Generating example 17
Generating example 18
Generating example 19
Generating example 20
Generating example 21
Generating example 22
Generating example 23
Generating example 24
Generating example 25
Generating example 26
Generating example 27
Generating example 28
Generating example 29
Generating example 30
Generating example 31
Generating example 32
Generating example 33
Generating example 34
Generating example 35
Generating example 36
Generating example 37
Generating example 38
Generating example 39
Generating example 40
Generating example 41
Generating example 42
Generating example 43
Generating example 44
Generating example 4

We also need to generate a system message.

In [4]:
def generate_system_message(prompt):

    prompt = 'A model that takes in a puzzle-like reasoning-heavy and specific questions in English for a college named Claremont Graduate University (CGU), and responds with a well-reasoned, accurate, step-by-step thought out response in English.'
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message.content

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given a complex and specific question related to Claremont Graduate University, provide a detailed and well-reasoned response with step-by-step explanations.`. Feel free to re-run this cell if you want a better result.


Now let's put our examples into a dataframe and turn them into a final pair of datasets.

In [17]:
!pip install pandas  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 9.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 8.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 11.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 11.4 MB/s eta 0:00:00


In [5]:
import json
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[0].strip())  # Corrected index for prompt
    responses.append(split_example[1].strip())  # Corrected index for response
  except IndexError as e:
    print(f"Error processing example: {example}. Error: {e}")
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples.')

# Initialize list to store training examples
training_examples = []

# Create training examples in the format required for GPT-3.5 fine-tuning
for index, row in df.iterrows():
    training_example = {
        "messages": [
            {"role": "system", "content": system_message.strip()},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['response']}
        ]
    }
    training_examples.append(training_example)

# Save training examples to a .jsonl file
with open('training_examples.jsonl', 'w') as f:
    for example in training_examples:
        f.write(json.dumps(example) + '\n')

There are 156 successfully-generated examples.


In [6]:
jsondata = pd.read_json('/Users/Parzon/Downloads/GPT GAN/training_examples.jsonl', lines =  True)

In [8]:
# Adjust display settings
pd.set_option('display.max_rows', 500)  # Adjust as needed
pd.set_option('display.max_columns', 10)  # Adjust as needed
pd.set_option('display.max_colwidth', None)  # Show full content of each field
pd.set_option('display.width', 1000)  # Adjust the total width of the display

# Upload the file to OpenAI

In [ ]:
file_id = openai.File.create(
  file=open("/content/training_examples.jsonl", "rb"),
  purpose='fine-tune'
).id

# Train the model! You may need to wait a few minutes before running the next cell to allow for the file to process on OpenAI's servers.

In [ ]:
job = openai.FineTuningJob.create(training_file=file_id, model="gpt-3.5-turbo")

job_id = job.id

# Now, just wait until the fine-tuning run is done, and you'll have a ready-to-use model!

Run this cell every 20 minutes or so -- eventually, you'll see a message "New fine-tuned model created: ft:gpt-3.5-turbo-0613:xxxxxxxxxxxx"

Once you see that message, you can go to the OpenAI Playground (or keep going to the next cells and use the API) to try the model!

In [ ]:
#openai.FineTuningJob.list_events(id=job_id, limit=10)

In [ ]:
# model_name_pre_object = openai.FineTuningJob.retrieve(job_id)
# model_name = model_name_pre_object.fine_tuned_model
# print(model_name)

In [10]:
job_id = 'ftjob-JnFgOwQ2F6D6glwBJHqCYOUA'

In [12]:
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.retrieve(job_id)


FineTuningJob(id='ftjob-JnFgOwQ2F6D6glwBJHqCYOUA', created_at=1711858412, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-1106:personal:cgu:98h873u8', finished_at=1711859442, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-Zv99cwny8iPo0sPegzKlrR6b', result_files=['file-XTcWFtFxojLLi1Iyfp0M7V2a'], status='succeeded', trained_tokens=42150, training_file='file-DLfFTxOJqaSy6oPXEN0J6Vgl', validation_file=None, user_provided_suffix='CGU')

# Once your model is trained, run the next cell to grab the fine-tuned model name.

# Let's try it out!

In [ ]:
user_prompt = "What classes does Dr.Li teach, be sure"
#df['prompt'].sample().values[0] 

# Print the user's prompt
print(f"User's prompt: {user_prompt}")

response = openai.chat.completions.create(
    model="ft:gpt-3.5-turbo-1106:personal:cgu:98h873u8",
    messages=[
      {
        "role": "system",
        "content": system_message,
      },
      {
          "role": "user",
          "content": user_prompt,
      }
    ],
)


print(response.choices[0].message.content)

User's prompt: What clasd does Dr.Li teach, be sure, shes in data science
Dr. Li teaches courses in the data science program, including "Data Analysis and Visualization" and "Machine Learning for Data Science."
